<td>
   <a target="_blank" href="https://labelbox.com" ><img src="https://labelbox.com/blog/content/images/2021/02/logo-v4.svg" width=256/></a>
</td>

<td>
<a href="https://colab.research.google.com/github/Labelbox/labelbox-python/blob/develop/extras/html_api_custom_editor.ipynb" target="_blank"><img
src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"></a>
</td>

<td>
<a href="https://github.com/Labelbox/labelbox-python/tree/develop//extras/html_api_custom_editor.ipynb" target="_blank"><img
src="https://img.shields.io/badge/GitHub-100000?logo=github&logoColor=white" alt="GitHub"></a>
</td>

# Create a custom editor for free text classifications using HTML assets
In this demo, we meant to demonstrate how you can create a custom editor using HTML assets in Labelbox using a custom HTML api designed by Labelbox. 
Please reach out to customer support if you want to have this api enabled in your organization

LB-HTML-API readme : https://storage.googleapis.com/labelbox-artifacts-public/dependencies/frontend/lb-html-api/0-0-2/README.md 

LB-HTML-JS : https://storage.googleapis.com/labelbox-artifacts-public/dependencies/frontend/lb-html-api/0-0-2/lb-html-api.js 



## Imports

In [ ]:
!pip install -q "labelbox[data]"


[notice] A new release of pip available: 22.3.1 -> 24.0
[notice] To update, run: pip install --upgrade pip


In [ ]:
import labelbox as lb
import urllib.request
import json
import json
import os
from string import Template
import uuid

# API Key and Client
See the developer guide for [creating an API key](https://docs.labelbox.com/reference/create-api-key).

In [ ]:
API_KEY = ""
client = lb.Client(api_key=API_KEY)

To get started we need to: 
-  Setup an HTML main template that will be embedded as an iframe in the html row data. The main template will contain the JavaScript that will interact with the Labelbox api 
- Setup a temporary template that will be use to generate the row data ( the data displayed in the labelbox editor) and use JSON data  to generate data row HTMLs
-  Upload HTMLs to Labelbox

## Step 1: Create an HTML template to use in the data row HTML iFrame
This template will interact directly with the Labelbox HTML api.

Currently we only support free text classifications payload

HTML API JavaScript supported Actions: 
  - `updateFreeText` : Updates a free text classification based on selections made in the data row in the HTML editor
  - `hideSideBar`: Hides the Labelbox classification right panel

Make sure that you are loading the JavaScript code at the top of your HTML document : https://storage.googleapis.com/labelbox-artifacts-public/dependencies/frontend/lb-html-api/0-0-2/lb-html-api.js 




For the purpose of this demo we have created an html template (see: <a href="view-source:https://storage.googleapis.com/labelbox-datasets/html_sample_data/html_api_demo/main_template.html" target="_blank">HTML SOURCE CODE</a> ) with all the JavaScript required to render the images and to interact with the Labelbox HTML API. 



## Step 2: Create a data row HTML template 
This HTML template will be used to generate each data row with all the corresponding row data. In this template we will insert the HTML template from above in an iFrame. 

You can follow the following steps to generate your data row html document. 
1. Create json data with all the data that will be presented in the editor.
2. Create a tempalte HTML document 
2. Transform the json files into html files 
3. upload html files into Labelbox 

In [ ]:
# Create one json data per data row
# this is an example of data that will be presented in the html iFrame
json_data = {"attribute": "Click on Photo and Select the Correct Flower Type",
  "gridImages": [
    {
      "imageId": "123M06700",
      "flowerType": "ORCHID",
      "imageTitle": "Plant Ecology - Ocean Level",
      "imageLocation": "San Andres, Colombia, SA",
      "imageDescription": "Flower blooming",
      "photoLink": "https://storage.googleapis.com/labelbox-datasets/image_sample_data/9378657435_89fabf13c9_n.jpg"
    },
    {
      "imageId": "1234M2S57",
      "flowerType": "TULIP",
      "imageTitle": "Plant Ecology - Ocean Level",
      "imageLocation": "Cartagena, Colombia, SA",
      "imageDescription": "Flower blooming",
      "photoLink": "https://storage.googleapis.com/labelbox-datasets/image_sample_data/9646730031_f3d5014416_n.jpg"
    },
    {
      "imageId": "1234567910",
      "flowerType": "ROSE",
      "imageTitle": "Plant Ecology -  Low Altitude",
      "imageLocation": "Bogota, Colombia, SA",
      "imageDescription": "Flower blooming",
      "photoLink": "https://storage.googleapis.com/labelbox-datasets/image_sample_data/99306615_739eb94b9e_m.jpg"
    },
    {
      "imageId": "1KJ567910",
      "flowerType": "SUNFLOWER",
      "imageTitle": "Plant Ecology -  Mountain Altitude",
      "imageLocation": "Cali, Colombia, SA",
      "imageDescription": "Flower blooming",
      "photoLink": "https://storage.googleapis.com/labelbox-datasets/image_sample_data/9423755543_edb35141a3_n.jpg"
    },
    {
      "imageId": "12224910",
      "flowerType": "TULIP",
      "imageTitle": "Plant Ecology -  Low Altitude",
      "imageLocation": "Cali, Colombia, SA",
      "imageDescription": "Flower blooming",
      "photoLink": "https://storage.googleapis.com/labelbox-datasets/image_sample_data/9446982168_06c4d71da3_n.jpg"
    }
  ]
}

### Create a data row html template

For the purpose of this demo we have created the following html template see : <a ref="view-source:https://storage.googleapis.com/labelbox-datasets/html_sample_data/html_api_demo/template.html"> SOURCE CODE </a>

####  Transform your json data into html documents using the source code from above
This is an example of how you can process your json data into an HTML file: 

In [ ]:
def create_html_with_data(json_data, html_template, output_path):
    """
    json_data: Dictionary with all the data row data that needs to be render in the iFrame
    html_template: template hosted publicly in the cloud
    output_path: the path were you want to store the HTML temporarily. The resulting HTML needs to be hosted publicly
    """
    js_data = json.dumps(json_data)
    template = Template(html_template)
    updated_html = template.safe_substitute(listingsData=js_data)
    with open(output_path, 'w') as file:
        file.write(updated_html)

The HTML output should look like the following: <a ref="https://storage.googleapis.com/labelbox-datasets/html_sample_data/html_api_demo/file1.html">json data processed as HTML</a>
Note that the HTML must have the embeeded JavaScript HTML template created in step 1.


### Step 3:  Upload HTML data rows to Labelbox

In [ ]:
data = [
  {
    "row_data": "https://storage.googleapis.com/labelbox-datasets/html_sample_data/html_api_demo/file1.html",
    "global_key": "file-1"
   },
  {
    "row_data": "https://storage.googleapis.com/labelbox-datasets/html_sample_data/html_api_demo/file2.html",
    "global_key": "file-2"
  },
  {
    "row_data": "https://storage.googleapis.com/labelbox-datasets/html_sample_data/html_api_demo/file3.html",
    "global_key": "file-3"

  }
]

#### Setup dataset, ontology and project

In [ ]:
NAME = ""
dataset = client.create_dataset(name = NAME)
task = dataset.create_data_rows(data)
task.wait_till_done()

print(task.errors)
print(task.result)

# Create project and ontology
project_name = "Find Correct Flowers"

ontology_builder = lb.OntologyBuilder(
    classifications=[
        lb.Classification(
            class_type=lb.Classification.Type.TEXT,
            name="Type")
    ]
)

ontology = client.create_ontology(
    project_name,
    ontology_builder.asdict(),
    media_type=lb.MediaType.Html)

project = client.create_project(
    name=project_name,
    media_type=lb.MediaType.Html)

project.setup_editor(ontology)

project.create_batches_from_dataset(name_prefix=f"flowers_dt_{uuid.uuid4()}",dataset_id=dataset.uid)